In [1]:
library(survey)
library(tidyverse)

Loading required package: grid
Loading required package: Matrix
Loading required package: survival

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_pco2_flag,lab_creatinine_flag,lab_potassium_flag,lab_troponin_flag,lab_po2_flag,lab_lactate_flag,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1,1,0,1,1,1,0,0.5525232,1.809879
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,1,1,0,0,0,0,1,0.2804613,1.389779
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1,1,0,1,1,1,0,0.3012020,3.320031
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,1,1,1,1,1,1,0,0.7982833,1.252688
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,1,1,0,0,0,0,0,0.2244447,1.289399
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,0,1,1,0,0,1,1,0,0.3320492,3.011602


In [4]:
wtd_pval <- data.table::fread(file.path(data_dir, "wtd_pval.csv"), data.table = FALSE)
head(wtd_pval)

cov,pval
age,0.076
gender,0.523
first_careunit,0.178
weight,0.163
elix_score,0.114
saps,0.134


In [5]:
wpv <- wtd_pval %>%
    filter(pval < 0.05) %>%
    pull(cov)

wpv

[1] "sofa"                       "vent"                      
 [3] "vaso"                       "sedative"                  
 [5] "icd_chf"                    "icd_afib"                  
 [7] "vs_cvp_flag"                "lab_chloride_first"        
 [9] "lab_bnp_flag"               "lab_troponin_flag"         
[11] "lab_creatinine_kinase_flag"

In [6]:
ipw_svydesign <- readRDS(file.path(data_dir, "ipw_svydesign.rds"))

In [7]:
fml <- wpv %>%
c("echo", .) %>%
paste(collapse = " + ") %>%
sprintf("mort_28_day ~ %s", .)

fml

[1] "mort_28_day ~ echo + sofa + vent + vaso + sedative + icd_chf + icd_afib + vs_cvp_flag + lab_chloride_first + lab_bnp_flag + lab_troponin_flag + lab_creatinine_kinase_flag"

In [8]:
logi <- svyglm(as.formula(fml), family = quasibinomial, design = ipw_svydesign)

In [9]:
summary(logi)


Call:
svyglm(formula = as.formula(fml), family = quasibinomial, design = ipw_svydesign)

Survey design:
svydesign(ids = ~icustay_id, weights = ~ps_weight, data = full_data)

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 -1.589674   0.488428  -3.255 0.001141 ** 
echo1                       -0.232787   0.070072  -3.322 0.000899 ***
sofa                         0.215857   0.011055  19.526  < 2e-16 ***
vent1                        0.394774   0.105845   3.730 0.000193 ***
vaso1                        0.156313   0.088181   1.773 0.076340 .  
sedative1                   -0.424582   0.105652  -4.019 5.92e-05 ***
icd_chf1                     0.072362   0.079327   0.912 0.361698    
icd_afib1                    0.358360   0.078084   4.589 4.53e-06 ***
vs_cvp_flag1                -0.433560   0.084130  -5.153 2.64e-07 ***
lab_chloride_first          -0.006852   0.004552  -1.505 0.132258    
lab_bnp_flag1                0.048641   0

In [10]:
exp(cbind(OR = coef(logi), confint(logi)))

,OR,2.5 %,97.5 %
(Intercept),0.2039921,0.07831824,0.5313291
echo1,0.7923226,0.69064827,0.9089651
sofa,1.2409250,1.21432690,1.2681057
vent1,1.4840481,1.20601500,1.8261786
vaso1,1.1691918,0.98361612,1.3897795
sedative1,0.6540429,0.53171011,0.8045212
icd_chf1,1.0750442,0.92024500,1.2558831
icd_afib1,1.4309802,1.22791630,1.6676253
vs_cvp_flag1,0.6481971,0.54966208,0.7643959
lab_chloride_first,0.9931712,0.98435053,1.0020709
